# [写真の表情・姿勢を操作するレシピ](https://axross-recipe.com/recipes/362)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# パッケージのインストール
!pip install PyYAML==5.1
!pip install torch==1.0.0
!pip install torchvision==0.2.1
!pip install imageio==2.3.0

In [ ]:
# gitからモデルをclone
!git init -q .
!git remote add origin https://github.com/AliaksandrSiarohin/first-order-model
!git pull -q origin master

In [ ]:
# パッケージのインポート
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# データのダウンロード（ここでエラーを吐いた場合は手作業でDL）
from google_drive_downloader import GoogleDriveDownloader as gdd

# 動画
gdd.download_file_from_google_drive(file_id='1PaOVqwPdnZkc5YzyqdL_ylCluave9xuU',
                                    dest_path='./04.mp4',
                                    unzip=False)
# 画像
gdd.download_file_from_google_drive(file_id='1GcG1g8IeJ0ooYqCO-s69S28fX68w5Rn7',
                                    dest_path='./02.png',
                                    unzip=False)

In [ ]:
# データの読込
source_image = imageio.imread('./02.png')
reader = imageio.get_reader('./04.mp4')

In [ ]:
# 画像のリサイズ
source_image = resize(source_image, (256, 256))[..., :3]

In [ ]:
# 動画のリサイズ
fps = reader.get_meta_data()['fps']
driving_video = []
try:
    for im in reader:
        driving_video.append(im)
except RuntimeError:
    pass
reader.close()

driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani

In [ ]:
# データの表示
HTML(display(source_image, driving_video).to_html5_video())

In [ ]:
# 学習済モデルのダウンロード
# vox-cpk.pth.tar を選択してダウンロード (https://drive.google.com/drive/folders/1PyQJmkdCsAkOYwUyaj_l-l0as-iLDgeH) 
# Gドライブの任意の場所に配置

In [ ]:
# 学習済モデルのコピー
!cp "/content/drive/MyDrive/Colab Notebooks/note_Axross/_data/vox-cpk.pth.tar" /content/

In [ ]:
# 学習済モデルのロード
from demo import load_checkpoints
generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', 
                            checkpoint_path='./vox-cpk.pth.tar')

In [ ]:
# アニメーションの生成
from demo import make_animation
from skimage import img_as_ubyte

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)

In [ ]:
# アニメーションの保存
imageio.mimsave('./generated.mp4', [img_as_ubyte(frame) for frame in predictions], fps=fps)

In [ ]:
# アニメーションの保存
HTML(display(source_image, driving_video, predictions).to_html5_video())